# Introduction

This notebook serves as an exploration of orbital debris modeling and an introduction to ODAP (Orbital Debris Analysis with Python). While ODAP is currently being refined and converted into a standalone module, you will be able to explore how the module functions and can be utilized in research throughout this notebook.

Initially, this project started from a personal curiosity about how orbital debris works, but later was the basis for a senior thesis written while attending the Harriet L. Wilkes Honors College. The purpose is to develop a modern open-source python implementation of the NASA Standard Breakup Model that others can use to research orbital debris.

Since this project is on going, please note that some functionality may not be working as expected as I continue to go through the process of optimizing and validating the implementations of the various components.

# Table of Contents
### 0. [Packages](#packages)
Covers the initial setup to enable the notebook to function correctly
### 1. [Data Source](#data-source)
Loading real world data from Two Line Elements to use as the foundation for the rest of the simulations
### 2. [Fragmentation Event Modeling](#fragmentation)
### 3. [Cloud Formation and Propagation](#cloud)
### 4. [Analysis](#second-bullet)


<a class="anchor" id="packages"></a>
<h1>0. Packages</h1>

For the purposes of this notebook I will be using a variety of other common modules such as NuMpy, pandas, and Plotly. As such, the below cell will import all necessary modules, as well as import various components from ODAP that will be useful.

In [11]:
# System lib.
import sys
import os
import re
import datetime as datetime
from importlib import reload

# 3rd party lib.
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
import chart_studio

# User defined lib.
if not os.path.join(sys.path[0], "..") in sys.path:
    sys.path.insert(1, os.path.join(sys.path[0], ".."))


## 0.1 Authenticate with Plotly

In [12]:
# Retreiving API Keys from OS
PLOTLY_API_KEY = os.environ.get("wK2Ilky4B7zHdcxvbtWV")
PLOTYLY_USERNAME = os.environ.get("rhumphreys2017")

# Log into chart studio for uploading plots
chart_studio.tools.set_credentials_file(
    username=PLOTYLY_USERNAME, api_key=PLOTLY_API_KEY
)

<a class="anchor" id="data-source"></a>
<h1>1. Data Source</h1>

To perform accurate simulations, it is essential to have some real-world satellite data as the starting point for modeling the fragmentation event. Therefore, this notebook is a file containing NORAD Two-Line Element Sets (TLE) acquired from [CelesTrak](https://celestrak.com/NORAD/elements/). The rest of this section involves importing said data and discussing how the TLE data structure functions.

<h3>1.1 Loading TLE Data</h3>

TLE's are standardized data structures that contain the orbital elements used to describe Earth-orbiting objects for a given point in time. Most importantly, they are used to determine where a given object will be at any given time. Thus, it is a valuable tool for analyzing potential orbital collisions as well as tracking orbital debris.

To acquire the most recent information about all objects being tracked in Earth orbit is recommended to download the latest TLE data from CelesTrak. This file comes in the form of a `.txt` file that first must be parsed to use while programming. While it is possible to manually parse all of the data from the TLEs using [Regular expression operations](https://docs.python.org/3/library/re.html), for this notebook, I will be utilizing the `skyfield` python module as it has the built-in functionality to handle this.

In [13]:
# Opening the .txt file
from odap.models.tle import TLE

with open("data/3le.txt") as f:
    txt = f.read()

# Using regular expression to perform basic parsing of the 3le.txt file
# Returns and array of arrays where each subarray contains three strings
# corresponding to each line of th TLE
tles_data = np.array([tle for tle in re.findall("(.*?)\n(.*?)\n(.*?)\n", txt)])

# Convert name line of TLE to correct format
tles_data[:, 0] = [name[2:] for name in tles_data[:, 0]]

tles = [TLE(lines[0], lines[1], lines[2]) for lines in tles_data]

<h3>1.2 Select orbital object for analysis</h3>

Now that we have an array containing Satellite objects, we can utilize NumPy to find a satellite that satisfies whatever criterion we are looking for. For example, we may want to simulate a fragmentation event for a satellite with a low semi-major axis or a high eccentricity. To keep things simple, I selected a satellite using the name assigned to it by NORAD.

In [14]:
from odap.models.satellite import Satellite

# Constructing an array that contains the names of all the satellites in our dataset
# Note: The name formatting of all satellites starts with "0 ",
#       thus we slice the String to cut the 0.
norad = np.array([tle.norad for tle in tles])

# Search the name array for the index of desired satellite
i = np.argwhere(norad == "07924").flatten()[0]

# Retrieve that satellite object from the `satellites` array using the found index
tle = tles[i]

# # Compute the cartesian state vectors
sat = Satellite(tle)
tle.name

'NIMBUS 6'

<a class="anchor" id="fragmentation"></a>
# 2. Fragmentation Event Modeling

A satellite breakup model is a mathematical model used to describe the outcome of a satellite breakup due to an explosion or collision. A satellite breakup model should describe the size, area-to-mass (AM) ratio, and the ejection velocity of each fragment produced in the satellite breakup. The most easily accessible literature model is the [NASA Standard breakup model](https://www.sciencedirect.com/science/article/abs/pii/S0273117701004239). This model is implemented in ODAP in `generate_debris.py`.

In the following subsections, ODAP is utilized to simulate an explosion event and a collision event. Additionally, some information about how the NASA Standard Breakup Model works is provided. For additional information it is recomended to refer to ...

In [15]:

import numpy as np

from kesspy import Satellite, ExplosionEvent, run_explosion
from importlib.metadata import version

# Define the initial position, velocity, mass, characteristic length, and kind of a satellite.
# TODO: Check the units of the position and velocity vectors from sat.position and sat.velocity
pos = np.array(sat.position).astype(np.float32)  # position vector [km] relative to Earth's center
vel = np.array(sat.velocity).astype(np.float32)  # velocity vector [km/s]'

mass = sat.mass; # [kg]
characteristic_length = 0.1; # [m]

# Create a new satellite with the given parameters
sat_kesspy = Satellite(pos, vel, mass)
# Create a new explosion event with the satellite
event = ExplosionEvent(sat_kesspy, characteristic_length)

# Run the simulation with the explosion event
debris = run_explosion(event)

print(f"{debris.shape} Pieces of debris generated.")




(238, 7, 3) Pieces of debris generated.


## 2.2 Performing a Fragmentation Event

#### 2.1.4 Converting coordinates to Keplerian elements

In [16]:
import odap.coordinates as coords

rs = debris[:, 1, :].astype(np.float64)  # km
# vs are delta velcoities, thus we need to add the initial velocity of the satellite
vs = debris[:, 6, :].astype(np.float64) / 1e3 # km/s
vs = vs + np.array(sat.velocity).astype(np.float64)

char_les = debris[:, 2, :].astype(np.float64) # m
ams = debris[:, 3, :].astype(np.float64) # kg
areas = debris[:, 4, :].astype(np.float64) # m^2
masses = debris[:, 5, :].astype(np.float64) # kg

# # unit: [km^3 / s^2]
# mu_Earth = 3.986004418e5

vel_mags = np.linalg.norm(vs, axis=1)
print("Average Char Length:", np.mean(char_les[:, 0]))
print("Average ams:", np.mean(ams[:, 0]))
print("Average areas:", np.mean(areas[:, 0]))
print("Average masses:", np.mean(masses[:, 0]))
print("Average vel mag:", np.mean(vel_mags))
print("Max vel mag:", np.max(vel_mags))
print("Min vel mag:", np.min(vel_mags))


# Currently python-sbm returns values in m and m/s, thus we need to convert them

# unit: [km^3 / s^2]
mu_Earth = 3.986004418e5

debris_keplerian_state = coords.rv2coe_many(mu_Earth, rs, vs)







Average Char Length: 0.2288543749444124
Average ams: 0.23331770476955577
Average areas: 0.08019314253381885
Average masses: 0.8431158508849135
Average vel mag: 7.305129318690417
Max vel mag: 7.570252014262096
Min vel mag: 7.1607673976920285


## 2.2 Ring Formation

In [17]:
from odap.propagators import OrbitPropagator, null_perts

# Cleanup states to remove any fragments that would deorbit, given no perturbations
a = debris_keplerian_state[:, 0]
e = debris_keplerian_state[:, 1]

masses = debris[:, 5, 0]
areas = debris[:, 4, 0]

# Prune debris that would deorbit giv
radius_earth = 6371.07103  # km
periapsis = a * (1 - e)  # km


I = np.argwhere(periapsis - radius_earth > 0)

debris_keplerian_state = np.squeeze(debris_keplerian_state[I, :])
areas = np.squeeze(areas[I])
masses = np.squeeze(masses[I])

# # Propagate orbit for a period of time
perts = null_perts()
perts["aero"] = True
perts["J2"] = True

# Define orbital periods
T = 2 * np.pi * np.sqrt((a*1e3)**3 / mu_Earth)

op = OrbitPropagator(
    debris_keplerian_state,
    areas,
    masses,
    [0, np.ceil(min(T))], # 40 orbits of the longest period
    400, # Atleast 10 points per orbit
    perts=perts,
)
states = np.concatenate((debris_keplerian_state[np.newaxis, :, :], op.propagate_orbit()), axis=0)

times [0.000000e+00 4.000000e+02 8.000000e+02 ... 1.919276e+08 1.919280e+08
 1.919284e+08]


In [20]:
states.shape

(479823, 238, 6)

In [18]:
def states_to_cartesian(states):
    states_cartesian = np.empty((states.shape[0], 2, states.shape[1], 3))
    for i in range(states.shape[0]):
        states_cartesian[i, :, :, :] = coords.coe2rv_many(
            mu_Earth,
            states[i, :, 0],
            states[i, :, 1],
            states[i, :, 2],
            states[i, :, 3],
            states[i, :, 4],
            states[i, :, 5],
        )
    return states_cartesian



states_cartesian = states_to_cartesian(states)

In [21]:

num_debris_to_plot = 100

positions = states_cartesian[:, 0, :, :]

# Create frames for each time step
frames = [
    go.Frame(
        data=
            [
            go.Scatter3d(
                x=[states_cartesian[i, 0, j, 0]],
                y=[states_cartesian[i, 0, j, 1]],
                z=[states_cartesian[i, 0, j, 2]],
                mode="markers",
                marker=dict(size=2),
            )
            for j in range(num_debris_to_plot)
            ]
    ) for i in range(1000)
]
fig = go.Figure(
    data=frames[0]['data'],  # Start with the data of the first frame
    frames=frames
)

# Determine global min and max values for x, y, z across all timesteps and debris pieces
x_min, x_max = np.min(states_cartesian[:, 0, :, 0]), np.max(states_cartesian[:, 0, :, 0])
y_min, y_max = np.min(states_cartesian[:, 0, :, 1]), np.max(states_cartesian[:, 0, :, 1])
z_min, z_max = np.min(states_cartesian[:, 0, :, 2]), np.max(states_cartesian[:, 0, :, 2])

# Update layout to have fixed axis ranges
fig.update_layout(
    updatemenus=[dict(
        type="buttons", 
        showactive=False, 
        buttons=[dict(label="Play", 
                      method="animate", 
                      args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)])])],
    scene=dict(
        xaxis=dict(title="x [km]", range=[x_min, x_max]),
        yaxis=dict(title="y [km]", range=[y_min, y_max]),
        zaxis=dict(title="z [km]", range=[z_min, z_max]),
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=1.25, y=1.25, z=1.25))
    )
)

fig.show()


In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.axes(projection="3d")

# Grabbing random pieces to visualize
I = np.random.randint(0, states_cartesian.shape[2], 200)

print(states_cartesian.shape)
# Data for three-dimensional scattered points
zdata = states_cartesian[1, 0, :, 2]
xdata = states_cartesian[1, 0, :, 0]
ydata = states_cartesian[1, 0, :, 1]



# # draw sphere
r_earth = 6371
u, v = np.mgrid[0 : 2 * np.pi : 20j, 0 : np.pi : 10j]
x = np.cos(u) * np.sin(v) * r_earth
y = np.sin(u) * np.sin(v) * r_earth
z = np.cos(v) * r_earth
ax.plot_surface(x, y, z, color="r")

ax.scatter3D(xdata, ydata, zdata)

In [ ]:
# # 3d animation of the debris cloud
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.animation import FuncAnimation
# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")
# ax.set_xlim3d(-10000, 10000)
# ax.set_ylim3d(-10000, 10000)
# ax.set_zlim3d(-10000, 10000)

# zdata = states_cartesian[:, 0, 0:20, 2]
# xdata = states_cartesian[:, 0, 0:20, 0]
# ydata = states_cartesian[:, 0, 0:20, 1]

# # # draw sphere
# r_earth = 6371
# u, v = np.mgrid[0 : 2 * np.pi : 20j, 0 : np.pi : 10j]
# x = np.cos(u) * np.sin(v) * r_earth
# y = np.sin(u) * np.sin(v) * r_earth
# z = np.cos(v) * r_earth
# ax.plot_surface(x, y, z, color="r")

# i = 10  # Change this to the desired index

# # Initialize scatter plot with first frame of data for the i-th debris fragment
# scatter = ax.scatter3D(states_cartesian[:5, 0, i, 0], 
#                        states_cartesian[:5, 0, i, 1], 
#                        states_cartesian[:5, 0, i, 2], 
#                        c=[states_cartesian[:5, 0, i, 2]], cmap="Greens")

# # Update function for animation of the i-th debris fragment
# def update(num):
#     scatter._offsets3d = ([states_cartesian[num, 0, i, 0]], 
#                           [states_cartesian[num, 0, i, 1]], 
#                           [states_cartesian[num, 0, i, 2]])
#     scatter.set_array([states_cartesian[num, 0, i, 2]])
#     return scatter,

# # Create animation
# ani = FuncAnimation(fig, update, frames=len(states_cartesian), blit=False)

# plt.show()




## 2.3 Band Formation

### 2.3.1 Drag Implementation

In [ ]:
import odap.aerodynamics as aero
import numpy as np


upper_bound = 900  # [km]
altitudes = np.arange(0, upper_bound, 1)  # [km]
rho = aero.atmosphere_density(altitudes)  # [kg·m^-3]

I_standard = np.argwhere(altitudes == 25).flatten()[0]
I_cira = np.argwhere(altitudes == 500).flatten()[0]

# Plotting the Exponential Atmospheric Model

layout = go.Layout(
    title=go.layout.Title(text="Altitude (z) vs. Atmospheric Density (ρ)", x=0.5),
    xaxis_title="z [km]",
    yaxis_title="$\log_{10}(\\rho\:[kg·m^{-3}])$",
    template="plotly_white",
    legend=go.layout.Legend(yanchor="top", y=0.99, xanchor="right", x=0.99),
)

data = [
    go.Scatter(
        x=altitudes[:I_standard],
        y=rho[:I_standard],
        mode="lines",
        name="U.S Standard Atmosphere",
    ),
    go.Scatter(
        x=altitudes[I_standard:I_cira],
        y=rho[I_standard:I_cira],
        mode="lines",
        name="CIRA-72",
    ),
    go.Scatter(
        x=altitudes[I_cira:],
        y=rho[I_cira:],
        mode="lines",
        name="CIRA-72 with T_infinity = 1000K",
    ),
]

fig = go.Figure(data=data, layout=layout)
fig.update_yaxes(type="log")


# fig.write_image("figures/Atmospheric_Density_v_Altitude.png", width=500, height=500, scale=2)
f2 = go.FigureWidget(fig)
f2

### 2.3.1 Applying Perturbations to Satellite

In [ ]:

op.tspan[-1] = 3600 * 24 * 365*30
op.dt = 3600 * 24

de, da, di, dOmega, domega, dnu, dp = op.propagate_perturbations()


In [ ]:
states_perts = np.stack((da, de, di, dOmega, domega, dnu))
states_perts = np.transpose(states_perts, (2, 1, 0))
states = np.concatenate((states, states_perts), axis=0)

In [ ]:
# Now that we have added on the perturbations, we need to recompute the cartesian states
states_cartesian = states_to_cartesian(states)

# 3. Analysis

## 3.1 Flux

In [ ]:
from skyfield.api import load
import matplotlib.dates as mdates
from dateutil import tz
import datetime as datetime

def fragmentation_flux(X):
    return np.sum((X[:-1, :, 1] < 0) & (X[1:, :, 1] > 0), axis=1)
    
position = states_cartesian[:, 0, :, :]
flux = fragmentation_flux(position)

w = 100 # Window of points to look at
tol = 5
convergence_ratio = np.array([np.var(flux[i:i+w])/np.mean(flux[i:i+w]) for i in range(len(flux))])    
intersection_index = np.argwhere(convergence_ratio <= tol).flatten()[0]

# datetimes
ts = load.timescale(builtin=True)
t_fragmentation = ts.now()
t_flux = t_fragmentation.utc_datetime() + np.array(range(len(flux))) * datetime.timedelta(minutes = 5)

# Removing last window from `t_flux`, `flux`, and `convergence_ratio` bc. not well defined for last values
t_flux = t_flux[:-w]
flux = flux[:-w]
convergence_ratio = convergence_ratio[:-w]

# Pruning data to the end of the ring formation
cs_toroid = states_cartesian[:intersection_index, :, :, :]
cs_toroid.shape
position.shape
# ks_toroid = op.inital_state[:intersection_index, :, :]
# op.states = ks_toroid

In [ ]:
np.linalg.norm(position[:, 0, :], axis=1)


### 3.1.1 FLux plot

In [ ]:
# Creating Flux v. Time plot
layout = go.Layout(
    title=dict(text="$\\text{Flux}\:(\\Phi)\:\\text{vs. Time }(t)$", x=0.5),
    xaxis_title="$t\:[days]$",
    yaxis_title="$\\text{ Number of fragments passing XZ plane, }\Phi\:$",
    template="plotly_white",
)


data = [
    go.Scatter(x=t_flux, y=flux, mode="lines", name="Flux"),
    go.Scatter(
        x=[t_flux[intersection_index], t_flux[intersection_index]],
        y=[0, np.max(flux)],
        mode="lines",
        line=dict(dash="dash"),
        name="Convergence",
    ),
]

fig1 = go.Figure(data=data, layout=layout)

# Stopping data to have half before intersection index and half after
# index_stop = intersection_index * 2
# if index_stop > len(flux) - 1:
#     index_stop = len(flux) - 1
# fig1.update_layout(xaxis_range=[t_flux[0], t_flux[index_stop]])

# Saving plot as an image and uploading it to plotly
# fig1.write_image("figures/Flux_v_Time.png", width=500, height=500, scale=2)
fig1.show()



### 3.1.2 Convergence Ratio plot

In [ ]:
# Creating Convergence Ratio v. Time plot
layout = go.Layout(
    title=dict(text="Convergence ratio vs. Time (t)", x=0.5),
    xaxis_title="$t\:[days]$",
    yaxis_title="Convergence ratio []",
    template="plotly_white",
    legend=go.layout.Legend(yanchor="top", y=0.99, xanchor="right", x=0.99),
)
data = [
    go.Scatter(x=t_flux, y=convergence_ratio, mode="lines", name="Convergence ratio"),
    go.Scatter(
        x=[t_flux[intersection_index], t_flux[intersection_index]],
        y=[0, np.max(flux)],
        mode="lines",
        line=dict(dash="dash"),
        name="Convergence time",
    ),
    go.Scatter(
        x=[t_flux[0], t_flux[-1]],
        y=[tol, tol],
        mode="lines",
        line=dict(dash="dash"),
        name="Tolerance",
    ),
]
fig2 = go.Figure(data=data, layout=layout)
fig2.update_yaxes(type="log")
fig2.write_image("figures/Convergence_Ratio_v_Time.png", width=500, height=500, scale=2)
# py.iplot(fig2, filename="Convergence Ratio v. Time")

## 3.2 Ring visualization

In [ ]:
import plotly.express as px
import plotly.io as pio
import pandas

spherical_earth_map = np.load("map_sphere.npy")

pos_toroid = cs_toroid[:, 0, :, :] / 1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb["radius"] / 1e3
xm, ym, zm = spherical_earth_map.T * r_E

# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5  # [s]
# Want to show the evolution in 30 min
timesteps = np.arange(0, N_timesteps, 6)

for t in timesteps:
    step = t * np.ones_like(N_timesteps)
    time = dt * step / 60  # [min]
    d = {
        "X": pos_toroid[t, :, 0],
        "Y": pos_toroid[t, :, 1],
        "Z": pos_toroid[t, :, 2],
        "Min.": time,
        # "a": ks_toroid[t, 0, :] / 1e3,
        # "e": ks_toroid[t, 1, :],
        # "i": ks_toroid[t, 2, :],
    }
    df = pandas.concat([df, pandas.DataFrame(data=d)])

# Creating visual
def spheres(size, clr, dist=0):

    # Set up 100 points. First, do angles
    theta = np.linspace(0, 2 * np.pi, 100)
    phi = np.linspace(0, np.pi, 100)

    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta), np.sin(phi))
    y0 = size * np.outer(np.sin(theta), np.sin(phi))
    z0 = size * np.outer(np.ones(100), np.cos(phi))

    # Set up trace
    trace = go.Surface(x=x0, y=y0, z=z0, colorscale=[[0, clr], [1, clr]])
    trace.update(showscale=False)

    return trace


fig = px.scatter_3d(
    data_frame=df,
    x="X",
    y="Y",
    z="Z",
    title="Evolution of debris cloud to toroid formation",
    hover_data={
        "Min.": False,
        "X": False,
        "Y": False,
        "Z": False,
        "a": ":.1f",
        "e": ":.4f",
        "i": ":.1f",
    },
    height=800,  # height of graph in pixels
    width=800,
    animation_frame="Min.",  # assign marks to animation frames
    range_x=[-r_E - 1000, r_E + 1000],
    range_z=[-r_E - 1000, r_E + 1000],
    range_y=[-r_E - 1000, r_E + 1000],
)
fig.update_traces(marker={"size": 3})
# Add Earth
earth = spheres(r_E, "#F0FFFF", 0)  # Earth
# fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig["layout"]["scene"]["aspectmode"] = "cube"
fig.add_trace(earth)
fig.update_layout(transition={"duration": 2000})
fig.write_html("plots/ring.html")

## 3.3 Band visualization

In [ ]:
temp = np.zeros_like(da)  # The params set to 0 dont matter for converting to rv
ks_propagated = np.swapaxes(
    np.stack([da, de, di, dOmega, domega, temp, dnu, dp, temp, temp]).T, 1, 2
)
ks_final = np.concatenate([ks_toroid, ks_propagated])
op.states = ks_final
cs_final = op.cartesian_representation()

In [ ]:
import pandas as pandas
import plotly.express as px

pos_toroid = cs_final[cs_toroid.shape[0] - 1 :, 0, :, :] / 1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb["radius"] / 1e3


# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5  # [s]
# Want to show the evolution in 1 day steps
timesteps = np.arange(0, N_timesteps, 5)

for t in timesteps:
    step = t * np.ones_like(N_timesteps)
    time = step  # [day]
    d = {
        "X": pos_toroid[t, :, 0],
        "Y": pos_toroid[t, :, 1],
        "Z": pos_toroid[t, :, 2],
        "Day": time,
    }
    df = pandas.concat([df, pandas.DataFrame(data=d)])


def spheres(size, clr, dist=0):

    # Set up 100 points. First, do angles
    theta = np.linspace(0, 2 * np.pi, 100)
    phi = np.linspace(0, np.pi, 100)

    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta), np.sin(phi))
    y0 = size * np.outer(np.sin(theta), np.sin(phi))
    z0 = size * np.outer(np.ones(100), np.cos(phi))

    # Set up trace
    trace = go.Surface(x=x0, y=y0, z=z0, colorscale=[[0, clr], [1, clr]])
    trace.update(showscale=False)

    return trace


fig = px.scatter_3d(
    data_frame=df,
    x="X",
    y="Y",
    z="Z",
    title="Evolution of debris cloud to Band formation",
    # labels={'Years in school (avg)': 'Years Women are in School'},
    # hover_data={'Min.': False, 'X': False, 'Y':False, 'Z':False, 'a':':.1f', 'e':':.4f','i':':.1f' },
    # hover_name='Orbital Elements',        # values appear in bold in the hover tooltip
    height=800,  # height of graph in pixels
    width=800,
    animation_frame="Day",  # assign marks to animation frames
    range_x=[-r_E - 1000, r_E + 1000],
    range_z=[-r_E - 1000, r_E + 1000],
    range_y=[-r_E - 1000, r_E + 1000],
)
fig.update_traces(marker={"size": 1.5, "color": "#6372f4"})
# Add Earth
earth = spheres(r_E, "#ffffff", 0)  # Earth
fig.add_trace(earth)
# fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig["layout"]["scene"]["aspectmode"] = "cube"
fig.update_layout(transition={"duration": 2000})
fig.update_layout(paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.write_html("plots/band.html")

## 3.4 Time to deorbit 

In [ ]:
import plotly.figure_factory as ff
import matplotlib.cm as cm

AM = op.A / op.M
z = (da * (1 - de)) - op.cb["radius"]
z[z < 100 * 1e3] = 0

layout = go.Layout(
    title=dict(text="Altitude of 50 debris fragments over 3 years", x=0.5),
    xaxis_title="$t\:[days]$",
    yaxis_title="Altitude [km]",
    template="plotly_white",
    legend=go.layout.Legend(yanchor="top", y=0.99, xanchor="right", x=0.99),
)

data = []

for i in range(25):
    alt = np.trim_zeros(z[i, :]) / 1e3
    scatter = go.Scatter(x=[i for i in range(len(alt))], y=alt, mode="lines")
    data.append(scatter)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(coloraxis=dict(colorscale="RdBu"), showlegend=False)
fig.show()
fig.write_image("figures/oxp_altitudes.png", width=500, height=500, scale=2)

## 3.5 Debris spread

In [ ]:
index = int(np.ceil(ks_propagated.shape[0] * 0.10))  # index near begining
raan_0 = ks_propagated[index, 3, :].copy() % 360
raan_0[raan_0 > 180] -= 360  # Converting angles to new range

raan_mid = ks_propagated[ks_propagated.shape[0] // 2, 3, :].copy() % 360
raan_mid[raan_mid > 180] -= 360

raan_f = ks_propagated[-1, 3, :].copy() % 360
raan_f[raan_f > 180] -= 360

In [ ]:
import math
import plotly.figure_factory as ff

uniform_dist = np.random.uniform(-180, 180, len(raan_0))
group_labels = [
    "$\Omega_{initial}$",
    "$\Omega_{midpoint}$",
    "$\Omega_{final}$",
    "uniform",
]
fig = ff.create_distplot(
    [raan_0, raan_mid, raan_f, uniform_dist], group_labels, show_hist=False
)

# Updating the uniform curve to be dashed
index = np.argwhere(np.array([data.legendgroup for data in fig.data]) == "uniform")[0][
    0
]
fig.data[index].line = dict(color="red", width=2, dash="dash")

# Layout
fig.layout["title"] = dict(text="Longitude of the ascending node distribution", x=0.5)
fig.layout["xaxis_title"] = "$\Omega\:[deg]$"
fig.layout["yaxis_title"] = "Kernel density estimation"
fig.layout["template"] = "plotly_white"


fig.write_image("figures/oxp_dist.png", width=500, height=500, scale=2)
py.iplot(fig, filename="Longitude of the ascending node distribution")

# 4. Conclusion